In [0]:
import os
CASSANDRA_IP=os.getenv('CASSANDRA1')
print(CASSANDRA_IP)

if CASSANDRA_IP is None:
    CASSANDRA_IP = '172.18.0.2'

from cassandra.cluster import Cluster
cluster = Cluster([CASSANDRA_IP])
session = cluster.connect()
session.execute('DROP KEYSPACE IF EXISTS classroom')
session.execute("CREATE KEYSPACE classroom WITH REPLICATION={'class':'SimpleStrategy', 'replication_factor':'1'}")
session = cluster.connect('classroom')
session.execute("create table student(id int PRIMARY KEY, firstname text, lastname text, emails set<text>)")
session.execute("insert into student (id, firstname, lastname, emails) values (1, 'Joe', 'Smith', {'joes@xyz.com', 'joe.smith@abc.net'})")
session.execute("update student set firstname = 'Joseph' where id = 1")
session.execute("insert into student (id, firstname, lastname, emails) values (2, 'Mike', 'Jones', {'mikej@xyz.com', 'mike.jones@def.net', 'mike1234@gmail.com'})")
rows = session.execute('SELECT id, firstname, lastname, emails from student')
print(list(rows))


### In order for Spark to talk to Cassandra, it needs to know the IP address to initialize the spark context with and it also needs the spark-cassandra-connector.

### Mongo will be similar and we need to initial the spark context with pointers to the mongo uri and also include the mongo-spark-connector.

### Additionally, whoever configures the cluster may need to make sure additional jars are installed in $SPARK_HOME/jars

Don't run the following, it is just included here to have a look at.

In [0]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.1,com.datastax.spark:spark-cassandra-connector_2.11:2.4.0 pyspark-shell'

def initspark(appname = "Test", servername = "local", cassandra="127.0.0.1", mongo="mongodb://127.0.0.1/classroom"):
    print ('initializing pyspark')
    conf = SparkConf().set("spark.cassandra.connection.host", cassandra).setAppName(appname).setMaster(servername)
    sc = SparkContext(conf=conf)
    spark = SparkSession.builder.appName(appname) \
    .config("spark.mongodb.input.uri", mongo) \
    .config("spark.mongodb.output.uri", mongo) \
    .enableHiveSupport().getOrCreate()
    sc.setLogLevel("WARN")
    print ('pyspark initialized')
    return sc, spark, conf


### Let's initialize as usual but pass in the IP address of the Cassandra cluster this time.

In [0]:
import sys
sys.path.append('/class')
from initspark import *
sc, spark, conf = initspark(cassandra=CASSANDRA_IP)


### Using the spark-cassandra-connector, we can read from a Cassandra table in a similar way to how we read from a MySQL table.

In [0]:
people = spark.read.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").load()
display(people)
print(people.collect())


### The results of a DataFrame can also be written to a Cassandra table.

In [0]:
# Append the results of a DataFrame into a Cassandra table
x = sc.parallelize([(3, 'Mary', 'Johnson', ['Mary1@gmail.com', 'Mary2@yahoo.com'])])
x1 = spark.createDataFrame(x, schema = ['id', 'firstname', 'lastname', 'emails'])
x1.write.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").mode("append").save()
print('Done')

In [0]:
people = spark.read.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").load()
display(people)


## LAB: ## 
### Read the shippers file from /class/datasets/northwind/JSON and save it to a Cassandra table in the classroom keyspace. Switch to a cqlsh window to prove it worked.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use printSchema to see the schema of the Spark DataFrame
<br>
The Cassandra table must exist before you can write to it, string in Spark is text in Cassandra, long is bigint in Cassandra
<br>
Build a create table command and executed it on the Cassandra session object
<br>
Write the DataFrame into the newly created table
<br>
<br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
shippers = spark.read.json('/class/datasets/northwind/JSON/shippers')
shippers.printSchema()
session.execute("create table if not exists shippers(companyname text, phone text, shipperid bigint PRIMARY KEY)")
shippers.write.format("org.apache.spark.sql.cassandra").options(table="shippers", keyspace="classroom").mode("append").save()

```
</p>
</details>

shippers = spark.read.json('/class/datasets/northwind/JSON/shippers')
shippers.printSchema()


### Once you have a DataFrame pointing to a Cassandra table, you can use normal SparkSQL on it by making it a temporary view. Here we will flatten out the list of emails using LATERAL VIEW EXPLODE.

In [0]:
people.createOrReplaceTempView('people')
people2 = spark.sql('select id, firstname, lastname, email from people LATERAL VIEW EXPLODE(emails) EXPLODED_TABLE AS email')
display(people2)


### Alternatively, you could also use dot syntax and make method calls.

In [0]:
people3 = people2.where("email like '%.com'").orderBy("id")
display(people3)

### Python directly talking to Mongo without Spark.

In [0]:
import pymongo
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
classroom = client["classroom"]
if 'classroom' in (x['name'] for x in client.list_databases()):
    client.drop_database('classroom')

people = classroom['people']
name = {"firstname" : "Adam", "personid":4}
x = people.insert_one(name)

names = [{"firstname" : "Betty", "personid":5}
         ,{"firstname" : "Charlie", "personid":6}]
x = people.insert_many(names)

x = people.find()
print ('*' * 80)
print ('from mongo directly')
print (list(x))
print ('*' * 80)



In [0]:
df = spark.read.format("mongo").option("uri", "mongodb://127.0.0.1/classroom.people").load()
df.show()


### Like Cassandra before, we can take a DataFrame and write it to a Mongo destination.

In [0]:
x = sc.parallelize([(7, 'David')])
x1 = spark.createDataFrame(x, schema = ['personid', 'firstname'])
x1.write.format("mongo").options(collection="people", database="classroom").mode("append").save()
print('Done')


In [0]:
df = spark.read.format("mongo").option("uri", "mongodb://127.0.0.1/classroom.people").load()
df.show()


### Like any DataFrame, we can make it into a temporary view and use SparkSQL on it.

In [0]:
df.createOrReplaceTempView('people')
spark.sql('select * from people').show()


## LAB: ## 
### Write shippers to Mongo and find all the shippers with an 800 number using  a temporary view.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Unlike Cassandra, Mongo does not require a collection to exist before writing to it, so just write the DataFrame to a new collection
<br>
Make a DataFrame from the new Mongo collection and turn it into a temporary view
<br>
Use SQL-like expression to find the desired records
<br>
<br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
shippers.write.format("mongo").options(collection="shippers", database="classroom").mode("append").save()

s=spark.read.format("mongo").option("uri","mongodb://127.0.0.1/classroom.shippers").load()
s.createOrReplaceTempView('shippers')
display(spark.sql("select * from shippers where phone like '%800%'"))
```
</p>
</details>

## HOMEWORK: ## 
**First Challenge**

Read Products from any source and write it to a Cassandra table. For simplicity, we only need to keep the productid, productname, and unitprice columns.

**Second Challenge**

Read Orders_LineItems.json from Day3 folder and write it to a Mongo collection.

**Third Challenge**

Join the Products and Orders_LineItems and join then, flatten them and regroup them so that the orders are grouped under each product instead.

**Bonus Challenge**

Include a calculated column showing how many times each product was ordered.

A starting template has been provided in Day4-Homework.py to deal with preparing the Cassandra and Mongo environments for Challenges 1 & 2. If you have difficulty doing those on your own, then start with that template. Otherwise, try it from scratch from the code provided in the course so far.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
<br>
Read each table from the NoSQL source and turn it into a temporary view
<br>
Use LATERAL VIEW EXPLODE() EXPLODED_TABLE to flatten out the nested format file or orders
<br>
Use the flattened results to join to products
<br>
Use the results of the join to group on productid, productname, and collect a structured list of customerid, orderid, orderdate, productid, quantity and price
<br>
Use the size function on the collected list to determine how many times each product was ordered or alternatively do it as part of the SQL query with other familiar techniques
<br>
<br>
</p>
</details>



